In [20]:
import pandas as pd
from datetime import datetime
import calendar

In [21]:
data = pd.read_csv(r'D:\Code\DS108\preprocessing\tidy_data.csv', encoding = 'utf-8-sig')

In [22]:
# Số dòng và cột của dữ liệu
data.shape

(13224, 30)

In [23]:
# Các thuộc tính của dữ liệu
data.columns

Index(['name', 'brand', 'condition', 'price_old', 'price_new', 'warranty',
       'CPU', 'RAM', 'capacity', 'time', 'battery', 'screen_size',
       'operating_system', 'display_technology', 'screen_resolution', 'SIM',
       'size', 'weight', 'bluetooth', 'refresh_rate', 'GPU', 'color', 'image',
       'resolution_width', 'resolution_height', 'has_nano_sim', 'has_esim',
       'height', 'width', 'depth'],
      dtype='object')

# Tính khoảng thời gian điện thoại đã ra mắt

In [24]:
data['time'] = pd.to_datetime(data['time'])

In [25]:
# Tính toán khoảng thời gian hiện tại - 'time'
data['time_difference'] = datetime.now() - data['time']

In [26]:
data.drop(columns=['time'], inplace=True)

# Tính thời gian hết bảo hành

In [27]:
processed_results = []
current_date = datetime.now() # Lấy ngày và giờ hiện tại

for index, item in data['warranty'].items():
    item = str(item).strip() # Đảm bảo item là chuỗi và loại bỏ khoảng trắng

    # Kiểm tra nếu có chứa "tháng"
    if "tháng" in item:
        num_months_str = item.replace("tháng", "").strip()
        num_months = int(num_months_str)

        # Xử lý cộng tháng
        year = current_date.year
        month = current_date.month + num_months
        day = current_date.day

        while month > 12:
            month -= 12
            year += 1
                
        # Xử lý trường hợp ngày không hợp lệ sau khi cộng tháng
        try:
            new_date = datetime(year, month, day)
        except ValueError:
            # Nếu ngày không hợp lệ, lấy ngày cuối cùng của tháng đó
            last_day_of_month = calendar.monthrange(year, month)[1]
            new_date = datetime(year, month, last_day_of_month)

        # Cập nhật giá trị
        data.loc[index, 'warranty'] = new_date.strftime("%d/%m/%Y")
    else:
            datetime.strptime(item, "%d/%m/%Y")
            data.loc[index, 'warranty'] = item 

# Chuyển dữ liệu sang kiểu datetime
data['warranty'] = pd.to_datetime(data['warranty'], format='%d/%m/%Y', errors='coerce')

# Xử lí dữ liệu kiểu object

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13224 entries, 0 to 13223
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   name                13224 non-null  object         
 1   brand               13224 non-null  object         
 2   condition           13224 non-null  object         
 3   price_old           13224 non-null  int64          
 4   price_new           13224 non-null  float64        
 5   warranty            13224 non-null  datetime64[ns] 
 6   CPU                 13193 non-null  object         
 7   RAM                 13036 non-null  float64        
 8   capacity            13204 non-null  float64        
 9   battery             13224 non-null  float64        
 10  screen_size         13224 non-null  float64        
 11  operating_system    11695 non-null  object         
 12  display_technology  12912 non-null  object         
 13  screen_resolution   13163 non-n

In [29]:
for col in data.columns:
    print(data[col].unique())

['iPhone 14 Pro Max' 'iPhone 13 Pro Max' 'OPPO Find N3 Flip'
 'Samsung Galaxy S24 Plus' 'Samsung Galaxy S24 Ultra' 'iPhone 14 Pro'
 'iPhone 15 Plus' 'Samsung Galaxy S23 Ultra' 'OPPO A78' 'OPPO Find N3'
 'iPhone 15 Pro' 'iPhone 15 Pro Max' 'Samsung Galaxy S23' 'iPhone 16 Plus'
 'iPhone 11' 'iPhone 8 Plus' 'Samsung Galaxy S22 Ultra' 'iPhone XS Max'
 'iPhone 12 Pro Max' 'Samsung Galaxy Z Fold5' 'iPhone 7 Plus'
 'Samsung Galaxy M55' 'iPhone 14' 'iPhone 16 Pro Max' 'iPhone 15'
 'iPhone 14 Plus' 'iPhone 13' 'Samsung Galaxy A35 5G' 'iPhone 12 Pro'
 'vivo Y18S' 'iPhone 16 Pro' 'realme Note 60' 'Xiaomi Redmi Note 14 5G'
 'iPhone 13 mini' 'iPhone 16' 'iPhone 11 Pro' 'iPhone X' 'iPhone 8'
 'iPhone 13 Pro' 'vivo V27e' 'iPhone 12' 'iPhone XS' 'iPhone 12 mini'
 'Samsung Galaxy A56 5G' 'iPhone SE' 'Xiaomi 13' 'Samsung Galaxy A55 5G'
 'Samsung Galaxy Z Fold4' 'OPPO Find X8' 'Xiaomi Poco X6 Pro 5G'
 'Samsung Galaxy S21 FE' 'iPhone 13 VN/A' 'Samsung Galaxy A73 5G'
 'Samsung Galaxy S24 FE 5G' 'Samsung Ga